# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

In [17]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2016-01-06,Current challenges facing the translation of b...,Frontiers in Cellular Neuroscience,This is a review of the challenges facing the ...,"Murphy, M. D., Guggenmos, D. J., Bundy, D. T.,...",2016-Frontiers-Review,https://fjfsdata01prod.blob.core.windows.net
1,2017-01-09,Comprehensive spatial analysis of the Borrelia...,Journal of Bacteriology,This paper is about the localization of lipopr...,"Dowdell, A. S., Murphy, M. D., Azodi, C., Swan...",2017-Borrelia-JBacteriol,https://jb.asm.org/content/jb/199/6/e00658-16....
2,2018-05-27,Assessing Perturbations to Neural Spiking Resp...,International Symposium on Circuits and Systems,This paper is about a proof-of-concept nonline...,"Murphy, M. D., Dunham, C., Nudo, R. J., Guggen...",2018-ISCAS-LFADS,https://ieeexplore.ieee.org/iel7/8334884/83508...
3,2019-10-29,Improving an open-source commercial system to ...,Journal of Neural Engineering,We modified the Intan Stimulation/Recording co...,"Murphy, M., Buccelli, S., Bornat, Y., Bundy, D...",2019-JNE-Intan-FSM,http://m053m716.github.io/files/Murphy_et_al__...
4,2019-10-30,Chronic stability of single-channel neurophysi...,PLoS ONE,Rats were trained on a 'gross' and 'fine' fore...,"Bundy, D. T., Guggenmos, D. J., Murphy, M. D.,...",2019-PLoS-Bundy,https://journals.plos.org/plosone/article/file...
5,2020-05-14,Differential effects of open-and closed-loop i...,Cerebral cortex,Intracortical microstimulation (ICMS) was appl...,"Averna, A., Pasquale, V., Murphy, M. D., Rogan...",2020-CerCor-Averna,https://watermark.silverchair.com/bhz281.pdf?t...
6,2020-11-01,Effects of tDCS on spontaneous spike activity ...,Brain Stimulation,We tested whether transcranial direct-current ...,"Milighetti, S., Sterzi, S., Fregni, F., Hanlon...",2020-Millighetti-tDCS,https://www.sciencedirect.com/science/article/...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.